# Analisis de archivos ab1 y obtener secuencias consenso.
# Cladograma con secuencias obtenidas mediante blastn
información  
https://biopython.org/wiki/SeqIO  
http://biopython.org/wiki/ABI_traces

In [ ]:
from Bio import SeqIO, SeqRecord, Seq, AlignIO, pairwise2, SearchIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC, generic_dna
from collections import defaultdict
import matplotlib.pyplot as plt
from io import StringIO
from Bio.Align import AlignInfo
from Bio.SubsMat.MatrixInfo import blosum62
from Bio.Blast import NCBIWWW, NCBIXML

In [ ]:
def consenso2(sec1, sec2):
    if len(sec1)!=len(sec2):
        print("las secuencias no son de igual tamaño")
        return
    n=0
    secuencia= ""
    nucleotido=""
    for row in range(len(sec1)):
        n+=1
        if sec1[row]== sec2[row]:
            secuencia+= sec1[row]
            #print (sec1[row], end = "")
        else:
            if sec1[row]=="-" and sec2[row]!="-":
                nucleotido=sec2[row].lower()
            elif sec1[row]!="-" and sec2[row]=="-":
                nucleotido=sec1[row].lower()
            elif sec1[row]!= sec2[row]:
                if (sec1[row]=="A" and sec2[row]== "G") or (sec1[row]=="G" and sec2[row]== "A"):
                    nucleotido="R"
                elif (sec1[row]=="C" and sec2[row]== "T") or (sec1[row]=="T" and sec2[row]== "C"):
                    nucleotido="Y"
                elif (sec1[row]=="G" and sec2[row]== "C") or (sec1[row]=="C" and sec2[row]== "G"):
                    nucleotido="S"
                elif (sec1[row]=="A" and sec2[row]== "T") or (sec1[row]=="T" and sec2[row]== "A"):
                    nucleotido="W"
                elif (sec1[row]=="G" and sec2[row]== "T") or (sec1[row]=="T" and sec2[row]== "T"):
                    nucleotido="K"
                elif (sec1[row]=="A" and sec2[row]== "C") or (sec1[row]=="C" and sec2[row]== "A"):
                    nucleotido="M"
            else:
                print ("hay dos errores, posición",row ,sec1[row], sec2[row])
                secuencia += nucleotido
            #print ( nucleotido, end = "")
            secuencia+= nucleotido

        if n==93:
            #print()
            n=0
    return (secuencia)

In [ ]:
def graficado(x1,x2):
    plt.figure(figsize=(16,4))
    plt.plot(trace['DATA9'], color='blue')
    plt.plot(trace['DATA10'], color='red')
    plt.plot(trace['DATA11'], color='green')
    plt.plot(trace['DATA12'], color='yellow')
    plt.xlim(x1,x2)  # se utiliza valores de 1000 como maximo
    plt.show()

In [ ]:
def generoespecie(genesp):
    genero=genesp[:genesp.find(" ")]
    print(genero)
    especie = genesp[genesp.find(" ")+1:]
    especie = especie[:especie.find(" ")]
    print(especie)
    genesp1 = genero+" "+especie
    return(genesp1)

In [ ]:
cd /Users/migueldelrio/Documents/2019Cicese/2019uam/cursosuam2019/pdi_molecular/secuencias/seq

In [ ]:
ls -lh *.ab1

# Para leer el archivo `ab1` se utiliza `SeqIO` con la opción `abi1`

In [ ]:
record = SeqIO.read('P1a-16S-F.ab1', 'abi')
record

# El archivo `ab1` contiene la información de la secuencia y del electroferograma.
## Graficado del electroferograma

In [ ]:
record.annotations.keys()
record.annotations['abif_raw'].keys()
channels = ['DATA9', 'DATA10', 'DATA11', 'DATA12']
trace = defaultdict(list)

In [ ]:
for c in channels:
    trace[c] = record.annotations['abif_raw'][c]

In [ ]:
graficado (0,7000)

In [ ]:
intervalos = range(1000,8000,1000)
intervalos

In [ ]:
for intervalo in intervalos:
    graficado (intervalo-1000,intervalo)

# El archivo `ab1` también contiene información sobre la calidad de la señal en cada nucleótido, por lo que se pueden quitar regiones de baja calidad de la secuencia de interés

# Eliminación de las regiones de baja calidad

# Se utilizarán ambas secuencias de un fragmento de interés.
### en este caso:
`P1a-16S-F.ab1`  
`P1a-16S-R.ab1`
# Secuencias con el iniciador sentido (forward) y antisentido (reverse)

In [ ]:
recordtrimmed_F = SeqIO.read('P1a-16S-F.ab1', 'abi-trim')
recordtrimmed_R = SeqIO.read('P1a-16S-R.ab1', 'abi-trim')

# Se alinean ambos fragmentos para corroborar la secuencia

In [ ]:
alignments = pairwise2.align.globalds(recordtrimmed_F.seq,recordtrimmed_R.seq.reverse_complement(), 
                                      blosum62, -10, -0.5)

# Se muestran los alineamientos obtenidos

In [ ]:
for row in range(len(alignments)):
    print ("Alineamiento", row)
    print(pairwise2.format_alignment(*alignments[row]), "\n")

# Seleccionar el mejor alineamiento en la siguiente celda como: 
`consenso2(alignments[x][0], alignments[x][1])`  
donde `x` es la secuencia seleccionada

### en este caso se selecciona la # `1`

In [ ]:
secuencia = consenso2(alignments[1][0], alignments[1][1])
print(secuencia)

# Las letras minúsculas indican que no hay nucleótidos en una de las dos secuencias
# Se utiliza el código IUPAC para designar variantes 

|Letra|nucleótidos|
|-----|-----------|
|R|A or G|
|Y|C or T|
|S|G or C|
|W|A or T|
|K|G or T|
|M|A or C|

# para los siguientes pasos, es necesario guardar la secuencia.
Explique qué es lo que hace el comando

In [ ]:
secuencia_fas = SeqRecord(Seq(secuencia.upper()), id="P1a-16S", description="P1a-16S" )

# Recuerde que un archivo fasta contiene
`> idendificador descriptor`  
`secuencia`

## Guardando la secuencia consenso con formato fasta

In [ ]:
SeqIO.write(secuencia_fas, 'P1a-16S_con.fasta', 'fasta')

# verificando contenido de `P1a-16S_con.fasta`

In [ ]:
!head P1a-16S_con.fasta

### Explique el contenido del archivo

In [ ]:
pwd

## Una vez obtenida la secuencia, se procede a buscarla en la bases de datos del NCBI, se puede hacer directamente en la página de blast o por búsqueda desde el `Jupyter`


In [ ]:
ls *.fasta

# Para buscar la secuencia en el NCBI, se usa NCBIWWW

### Se busca en la base de datos de nucleótidos, el formato del archivo es _fasta_ (`record.format`) y el número de secuencias que regresará es de 10 (por omisión son 50, `hitlist_size`)

In [ ]:
record = SeqIO.read("P1a-16S_con.fasta", format="fasta")
result_handle = NCBIWWW.qblast("blastn", "nt", record.format("fasta"), hitlist_size = 10)

Se guarda el archivo en formato `xml` y se cierra el archivo recibido (result_handle.close())

In [ ]:
with open("P1a.xml", "w") as out_handle:
    out_handle.write(result_handle.read())

result_handle.close()

In [ ]:
result_handle = open("P1a.xml")
blast_record = NCBIXML.read(result_handle)

### Si se desea un valor de corte definido (E_VALUE_THRESH) se ejecuta la siguiente celda

# Si no se desea un valor de corte, se ejecuta la siguiente celda para observar los resultados del blast:

In [ ]:
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        print("****Alignment****")
        print("sequence:", alignment.title)
        print("length:", alignment.length)
        print("e value:", hsp.expect)
        print(hsp.query[0:75] + "...")
        print(hsp.match[0:75] + "...")
        print(hsp.sbjct[0:75] + "...")

# Para guardar las secuencias resultantes del blast se ejecuta la siguiente celda. 
## Se almacenan las secuencias en la variable `secuencias`

In [ ]:
E_VALUE_THRESH = 0.001
secuencias = []
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
         if hsp.expect < E_VALUE_THRESH:
            print("****Alignment****")
            print("sequence:", alignment.title)
            print("length:", alignment.length)
            print("e value:", hsp.expect)
            print(hsp.query[0:75] + "...")
            print(hsp.match[0:75] + "...")
            print(hsp.sbjct[0:75] + "...")
            linea =SeqRecord(Seq(hsp.sbjct), id=alignment.accession , description=generoespecie(alignment.hit_def))
            secuencias.append(linea)

# Para anexar la secuencia sometida al blast se corrobora que la variable `record` tenga la información de la secuencia de interés 

In [ ]:
#record = SeqIO.read("P1a-16S_con.fasta", format="fasta")
record

### se agrega la secuencia de interés a la variable `secuencias`

In [ ]:
secuencias.append(record)

In [ ]:
SeqIO.write(secuencias, 'P1a_alineamiento.fasta', 
            'fasta')

In [ ]:
import os, pylab
from Bio.Align.Applications import ClustalwCommandline
from Bio import pairwise2, SeqIO, AlignIO, Phylo


## Alineamiento con `ClustalW` de las secuencias obtenidas con el blast y la de interés

In [ ]:
clustalw_exe = r"/Users/migueldelrio/Desktop/analisis/scripts/clustalw2"
clustalw_cline = ClustalwCommandline(clustalw_exe, infile="P1a_alineamiento.fasta")
assert os.path.isfile(clustalw_exe), "Clustal W executable missing"
stdout, stderr = clustalw_cline()

In [ ]:
ls

# Visualización de las secuencias alineadas

In [ ]:
alignments = AlignIO.parse("P1a_alineamiento.aln", "clustal")
for alignment in alignments:
    print(alignment)
    print("")

# Visualización del árbol con caracteres ASCII

In [ ]:
# para visualizar el árbol generado en formato ascii, se ve el contenido del archivo .dnd
tree = Phylo.read("P1a_alineamiento.dnd", "newick")
Phylo.draw_ascii(tree, file=None, column_width=80)

# Visualización del árbol dibujado

In [ ]:
tree.rooted = True
Phylo.draw(tree)